In [ ]:
# import header files
%matplotlib inline
import torch
import torch.nn as nn
import torchvision
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import glob
import os
import random
import tensorflow as tf
from tensorflow import keras
import numpy as np
import seaborn as sn
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import time
import copy
import tqdm
import torch
import random
from PIL import Image
import torch.optim as optim
from torchvision import models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset,DataLoader

In [ ]:
# load my google drive
def auth_gdrive():
  from google.colab import drive
  if os.path.exists('content/gdrive/My Drive'): return
  drive.mount('/content/gdrive')
def load_gdrive_dataset():
  loader_assets = 'myepill.zip'
  auth_gdrive()

In [ ]:
# mount my google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
load_gdrive_dataset()

Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# unzip dataset
!unzip "/content/gdrive/MyDrive/myepill.zip"

Streaming output truncated to the last 5000 lines.
  inflating: myepill/train/2/55154-3495_0_0.jpg  
  inflating: myepill/train/2/55154-3495_0_1.jpg  
  inflating: myepill/train/2/55154-3516_0_0.jpg  
  inflating: myepill/train/2/55154-3516_0_1.jpg  
  inflating: myepill/train/2/55154-3526_0_0.jpg  
  inflating: myepill/train/2/55154-3526_0_1.jpg  
  inflating: myepill/train/2/55154-3580_0_0.jpg  
  inflating: myepill/train/2/55154-3580_0_1.jpg  
  inflating: myepill/train/2/55154-3581_0_0.jpg  
  inflating: myepill/train/2/55154-3581_0_1.jpg  
  inflating: myepill/train/2/55154-3582_0_0.jpg  
  inflating: myepill/train/2/55154-3582_0_1.jpg  
  inflating: myepill/train/2/55154-3621_0_0.jpg  
  inflating: myepill/train/2/55154-3621_0_1.jpg  
  inflating: myepill/train/2/55154-3622_0_0.jpg  
  inflating: myepill/train/2/55154-3622_0_1.jpg  
  inflating: myepill/train/2/55154-3625_0_0.jpg  
  inflating: myepill/train/2/55154-3625_0_1.jpg  
  inflating: myepill/train/2/55154-3650_0_0.jpg  

In [ ]:
# define transforms
train_transforms = torchvision.transforms.Compose([torchvision.transforms.RandomRotation(30),
                                       torchvision.transforms.Resize((32, 32)),
                                       torchvision.transforms.RandomHorizontalFlip(),
                                       torchvision.transforms.ToTensor(),
                                       torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
# get data
train_data = torchvision.datasets.ImageFolder("/content/myepill/train/", transform=train_transforms)
test_data = torchvision.datasets.ImageFolder("/content/myepill/test/", transform=train_transforms)

In [ ]:
# data loader
trainloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=1, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True, num_workers=1, pin_memory=True)

In [ ]:
# define the model
import torch.nn as nn
import math


def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


def make_divisible(x, divisible_by=8):
    import numpy as np
    return int(np.ceil(x * 1. / divisible_by) * divisible_by)


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(inp * expand_ratio)
        self.use_res_connect = self.stride == 1 and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetv2(nn.Module):
    def __init__(self, n_class=1000, input_size=224, width_mult=1.):
        super(MobileNetv2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280
        interverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        assert input_size % 32 == 0
        # input_channel = make_divisible(input_channel * width_mult)  # first channel is always 32!
        self.last_channel = make_divisible(last_channel * width_mult) if width_mult > 1.0 else last_channel
        self.features = [conv_bn(3, input_channel, 2)]
        # building inverted residual blocks
        for t, c, n, s in interverted_residual_setting:
            output_channel = make_divisible(c * width_mult) if t > 1 else c
            for i in range(n):
                if i == 0:
                    self.features.append(block(input_channel, output_channel, s, expand_ratio=t))
                else:
                    self.features.append(block(input_channel, output_channel, 1, expand_ratio=t))
                input_channel = output_channel
        # building last several layers
        self.features.append(conv_1x1_bn(input_channel, self.last_channel))
        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # building classifier
        self.classifier = nn.Linear(self.last_channel, n_class)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.mean(3).mean(2)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


def mobilenet_v2(pretrained=True):
    model = MobileNetv2(width_mult=1)

    if pretrained:
        try:
            from torch.hub import load_state_dict_from_url
        except ImportError:
            from torch.utils.model_zoo import load_url as load_state_dict_from_url
        state_dict = load_state_dict_from_url(
            'https://www.dropbox.com/s/47tyzpofuuyyv1b/mobilenetv2_1.0-f2a8633.pth.tar?dl=1', progress=True)
        model.load_state_dict(state_dict)
    return model


if __name__ == '__main__':
    net = mobilenet_v2(True)


Downloading: "https://www.dropbox.com/s/47tyzpofuuyyv1b/mobilenetv2_1.0-f2a8633.pth.tar?dl=1" to /root/.cache/torch/hub/checkpoints/mobilenetv2_1.0-f2a8633.pth.tar
100%|██████████| 13.5M/13.5M [00:01<00:00, 13.1MB/s]


In [ ]:
# print the model
import math
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MobileNetv2()
model.to(device)

MobileNetv2(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
       

In [ ]:
# print summary of the model
from torchvision import models
from torchsummary import summary
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 16, 16]             864
       BatchNorm2d-2           [-1, 32, 16, 16]              64
             ReLU6-3           [-1, 32, 16, 16]               0
            Conv2d-4           [-1, 32, 16, 16]             288
       BatchNorm2d-5           [-1, 32, 16, 16]              64
             ReLU6-6           [-1, 32, 16, 16]               0
            Conv2d-7           [-1, 16, 16, 16]             512
       BatchNorm2d-8           [-1, 16, 16, 16]              32
  InvertedResidual-9           [-1, 16, 16, 16]               0
           Conv2d-10           [-1, 96, 16, 16]           1,536
      BatchNorm2d-11           [-1, 96, 16, 16]             192
            ReLU6-12           [-1, 96, 16, 16]               0
           Conv2d-13             [-1, 96, 8, 8]             864
      BatchNorm2d-14             [-1, 9

In [ ]:
# loss function to be used
criterion = torch.nn.CrossEntropyLoss()
# optimizer to be used
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, momentum=0.9, weight_decay=5e-4)

In [ ]:
# training step...
from torch.utils.tensorboard import SummaryWriter
train_losses = 0.0
train_accuracy = 0
epochs = 50
for epoch in range(epochs):  # loop over the dataset multiple times
    print('Epoch-{0}:'.format(epoch + 1, optimizer.param_groups[0]['lr']))
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data # get the inputs; data is a list of [inputs, labels]
        inputs, labels = inputs.cuda(), labels.cuda() # for using data in GPU
        optimizer.zero_grad() # zero the parameter gradients
        outputs = model(inputs) # forward
        loss = criterion(outputs, labels) # calculate loss
        loss.backward() # backward loss
        optimizer.step() # optimize gradients
        train_losses += loss.item() # save loss
        _, preds = torch.max(outputs, 1) # save prediction
        train_accuracy += torch.sum(preds == labels.data) # save train_accuracy       
        if i % 1000 == 999:    # every 1000 mini-batches...           
            steps = epoch * len(trainloader) + i # calculate steps 
            batch = i*batch_size # calculate batch 
            # Save train_accuracy and loss to Tensorboard
            writer.add_scalar('Training loss by steps', train_losses / batch, steps)
            writer.add_scalar('Training accuracy by steps', train_accuracy / batch, steps)          
    train_losses = 0.0
    train_accuracy = 0    
print('Train is finished...')

Epoch-1:
Epoch-2:
Epoch-3:
Epoch-4:
Epoch-5:
Epoch-6:
Epoch-7:
Epoch-8:
Epoch-9:
Epoch-10:
Epoch-11:
Epoch-12:
Epoch-13:
Epoch-14:
Epoch-15:
Epoch-16:
Epoch-17:
Epoch-18:
Epoch-19:
Epoch-20:
Epoch-21:
Epoch-22:
Epoch-23:
Epoch-24:
Epoch-25:
Epoch-26:
Epoch-27:
Epoch-28:
Epoch-29:
Epoch-30:
Epoch-31:
Epoch-32:
Epoch-33:
Epoch-34:
Epoch-35:
Epoch-36:
Epoch-37:
Epoch-38:
Epoch-39:
Epoch-40:
Epoch-41:
Epoch-42:
Epoch-43:
Epoch-44:
Epoch-45:
Epoch-46:
Epoch-47:
Epoch-48:
Epoch-49:
Epoch-50:
Train is finished...


In [ ]:
# test step...
from torch.utils.tensorboard import SummaryWriter
test_losses = 0.0
test_accuracy = 0
epochs = 50
for epoch in range(epochs):  # loop over the dataset multiple times
    print('Epoch-{0}:'.format(epoch + 1, optimizer.param_groups[0]['lr']))
    for i, data in enumerate(testloader, 0):
        inputs, labels = data # get the inputs; data is a list of [inputs, labels]
        inputs, labels = inputs.cuda(), labels.cuda() # for using data in GPU
        optimizer.zero_grad() # zero the parameter gradients      
        outputs = model(inputs) # forward
        loss = criterion(outputs, labels) # calculate loss
        loss.backward() # backward loss
        optimizer.step() # optimize gradients
        test_losses += loss.item() # save loss
        _, preds = torch.max(outputs, 1) # save prediction
        test_accuracy += torch.sum(preds == labels.data) # save test_accuracy        
        if i % 1000 == 999:    # every 1000 mini-batches...           
            steps = epoch * len(testloader) + i # calculate steps 
            batch = i*batch_size # calculate batch 
            print("Test loss {:.5} Test Accuracy {:.5} Steps: {}".format(test_losses / batch, test_accuracy/batch, steps))           
            # Save test_accuracy and loss to Tensorboard
            writer.add_scalar('Test loss by steps', test_losses / batch, steps)
            writer.add_scalar('Test accuracy by steps', test_accuracy / batch, steps)                      
    print("Test Accuracy: {}/{} ({:.5} %) Test Loss: {:.5}".format(test_accuracy, len(testloader), 100. * test_accuracy / len(testloader.dataset), test_losses / len(testloader.dataset)))   
    test_losses = 0.0
    test_accuracy = 0    
print('Test is Finished...')

Epoch-1:
Test Accuracy: 881/125 (88.1 %) Test Loss: 0.050782
Epoch-2:
Test Accuracy: 936/125 (93.6 %) Test Loss: 0.025268
Epoch-3:
Test Accuracy: 931/125 (93.1 %) Test Loss: 0.024174
Epoch-4:
Test Accuracy: 941/125 (94.1 %) Test Loss: 0.023317
Epoch-5:
Test Accuracy: 954/125 (95.4 %) Test Loss: 0.017802
Epoch-6:
Test Accuracy: 962/125 (96.2 %) Test Loss: 0.012948
Epoch-7:
Test Accuracy: 963/125 (96.3 %) Test Loss: 0.013608
Epoch-8:
Test Accuracy: 964/125 (96.4 %) Test Loss: 0.012869
Epoch-9:
Test Accuracy: 973/125 (97.3 %) Test Loss: 0.010396
Epoch-10:
Test Accuracy: 974/125 (97.4 %) Test Loss: 0.012222
Epoch-11:
Test Accuracy: 971/125 (97.1 %) Test Loss: 0.01047
Epoch-12:
Test Accuracy: 980/125 (98.0 %) Test Loss: 0.0086845
Epoch-13:
Test Accuracy: 972/125 (97.2 %) Test Loss: 0.010664
Epoch-14:
Test Accuracy: 979/125 (97.9 %) Test Loss: 0.0077755
Epoch-15:
Test Accuracy: 977/125 (97.7 %) Test Loss: 0.006912
Epoch-16:
Test Accuracy: 970/125 (97.0 %) Test Loss: 0.010477
Epoch-17:
Test A